In [31]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif

In [38]:
df = pd.read_csv('bank-full.csv', sep=';')

In [39]:
#Проверка на пропуски
df.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [40]:
#Вопрос 1: Mode для столбца education
df['education'].mode()

0    secondary
Name: education, dtype: object

In [41]:
#Вопрос 2: Корреляционная матрица
corr_matrix = df[['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']].corr()
print(corr_matrix)

               age   balance       day  duration  campaign     pdays  previous
age       1.000000  0.097783 -0.009120 -0.004648  0.004760 -0.023758  0.001288
balance   0.097783  1.000000  0.004503  0.021560 -0.014578  0.003435  0.016674
day      -0.009120  0.004503  1.000000 -0.030206  0.162490 -0.093044 -0.051710
duration -0.004648  0.021560 -0.030206  1.000000 -0.084570 -0.001565  0.001203
campaign  0.004760 -0.014578  0.162490 -0.084570  1.000000 -0.088628 -0.032855
pdays    -0.023758  0.003435 -0.093044 -0.001565 -0.088628  1.000000  0.454820
previous  0.001288  0.016674 -0.051710  0.001203 -0.032855  0.454820  1.000000


In [42]:
#Наибольшая корреляция у pdays и previous = 0.454820

In [43]:
#Вопрос 3: Взаимная информация
features = ['age', 'job', 'marital', 'education', 'balance', 'housing', 
            'contact', 'day', 'month', 'duration', 'campaign', 
            'pdays', 'previous', 'poutcome']
df['y'] = df['y'].apply(lambda x: 1 if x == 'yes' else 0)
X = df.drop(columns=['y'])
y = df['y']

# Применение one-hot кодирования к категориальным переменным
X_encoded = pd.get_dummies(X, drop_first=True)

# Разделение данных на тренировочный и тестовый наборы
X_train, X_val, y_train, y_val = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Вычисление взаимной информации
mi_scores = mutual_info_classif(X_train, y_train, discrete_features=True)
mi_scores_series = pd.Series(mi_scores, index=X_train.columns)
print("Взаимная информация:\n", mi_scores_series.round(2))

Взаимная информация:
 age                    0.01
balance                0.10
day                    0.01
duration               0.10
campaign               0.00
pdays                  0.03
previous               0.01
job_blue-collar        0.00
job_entrepreneur       0.00
job_housemaid          0.00
job_management         0.00
job_retired            0.00
job_self-employed      0.00
job_services           0.00
job_student            0.00
job_technician         0.00
job_unemployed         0.00
job_unknown            0.00
marital_married        0.00
marital_single         0.00
education_secondary    0.00
education_tertiary     0.00
education_unknown      0.00
default_yes            0.00
housing_yes            0.01
loan_yes               0.00
contact_telephone      0.00
contact_unknown        0.01
month_aug              0.00
month_dec              0.00
month_feb              0.00
month_jan              0.00
month_jul              0.00
month_jun              0.00
month_mar              0.0

In [46]:
#Вопрос 4: Логистическая регрессия
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

accuracy = accuracy_score(y_val, model.predict(X_val))
print("Точность на валидационном наборе:", round(accuracy, 2))

Точность на валидационном наборе: 0.9


In [47]:
#Вопрос 5: Исключение признаков
initial_accuracy = accuracy

for col in X_train.columns:
    X_train_reduced = X_train.drop(columns=[col])
    X_val_reduced = X_val.drop(columns=[col])
    
    model.fit(X_train_reduced, y_train)
    y_pred_val_reduced = model.predict(X_val_reduced)
    accuracy_reduced = accuracy_score(y_val, y_pred_val_reduced)
    
    print(f'Признак: {col}, Разница в точности: {round(initial_accuracy - accuracy_reduced, 4)}')

Признак: age, Разница в точности: 0.0007
Признак: balance, Разница в точности: 0.0004
Признак: day, Разница в точности: -0.0002
Признак: duration, Разница в точности: 0.0096
Признак: campaign, Разница в точности: 0.0003
Признак: pdays, Разница в точности: 0.0007
Признак: previous, Разница в точности: -0.0001
Признак: job_blue-collar, Разница в точности: -0.0002
Признак: job_entrepreneur, Разница в точности: -0.0001
Признак: job_housemaid, Разница в точности: -0.0001
Признак: job_management, Разница в точности: -0.0002
Признак: job_retired, Разница в точности: 0.0002
Признак: job_self-employed, Разница в точности: 0.0008
Признак: job_services, Разница в точности: -0.0007
Признак: job_student, Разница в точности: 0.0008
Признак: job_technician, Разница в точности: -0.0004
Признак: job_unemployed, Разница в точности: 0.0008
Признак: job_unknown, Разница в точности: 0.0006
Признак: marital_married, Разница в точности: 0.0001
Признак: marital_single, Разница в точности: -0.0003
Признак: edu

In [48]:
#Наименьшая разница у previous = -0.0001

In [52]:
#Вопрос 6: Регуляризованная логистическая регрессия 
C_values = [0.01, 0.1, 1, 10, 100]
best_accuracy = 0
best_C = 0
accuracies = []

for C in C_values:
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    accuracy = accuracy_score(y_val, model.predict(X_val))
    accuracies.append(accuracy)  # Сохраняем точность для каждого C
    print(f"Точность при C={C}: {round(accuracy, 4)}")
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_C = C

print("Наилучшее значение C:", best_C)

Точность при C=0.01: 0.8963
Точность при C=0.1: 0.898
Точность при C=1: 0.8996
Точность при C=10: 0.8995
Точность при C=100: 0.8996
Наилучшее значение C: 1
